## Commission Calculator


### Goal: Calculate the total commission earned by a sales rep for a given period of time
Author: Saba Sajjadi<br>
Date: 3/29/2023

### Importing required python packages

In [1]:
import pandas as pd
import json
from datetime import datetime

In order to create a commission calculator, I wrote two functions. The first one is called load_dataframes(), which is a helper function that checks if datasets were imported successfully. The second function is called total_commission(), which takes sales_rep_name, start_date, and end_date as inputs and returns the total commission amount based on the input criteria.

In [14]:
def load_dataframes():
    try:
        # Load the datasets as dataframes
        with open('Deals.json') as f:
            deals = pd.read_json(f)
        with open('Products.json') as f:
            products = pd.read_json(f)
        return deals, products
    except FileNotFoundError:
        print("One or both of the data files are missing.")
        return None, None


def total_commission(sales_rep_name, start_date, end_date):
    # Load the datasets as dataframes
    deals, products = load_dataframes() #calling the load_dataframes function 
    
    if deals is None or products is None:
        return None
    
    # Check if the sales rep name is valid
    if sales_rep_name not in deals['sales_rep_name'].unique():
        print(f"{sales_rep_name} is not a valid sales rep name.")
        return None
    
    # Check if the start date and end date are valid dates
    try:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    except ValueError:
        print("The start date or end date is not a valid date.")
        return None
    
    # Filter the deals dataframe based on the sales rep name and date range
    deals = deals[(deals['sales_rep_name'] == sales_rep_name) & (deals['date'] >= start_date) & (deals['date'] <= end_date)]

    # Merge the filtered deals dataframe with the products dataframe
    merged = pd.merge(deals, products, left_on='product_id', right_on='id', how='left')

    # Calculate the commission for each sale
    merged['commission'] = merged['quantity_products_sold'] * merged['product_amount'] * merged['commission_rate']

    # Group the merged dataframe by sales rep name and sum the commissions
    result = merged.groupby('sales_rep_name')['commission'].sum().get(sales_rep_name, 0)

    return result


In [15]:
result = total_commission(sales_rep_name="Ian", start_date="2023-01-01", end_date="2023-04-30")
print(f"total_commission(sales_rep_name='Ian', start_date='2023-01-01', end_date='2023-04-30') = {result:.2f}")

total_commission(sales_rep_name='Ian', start_date='2023-01-01', end_date='2023-04-30') = 55350.00


In [16]:
# Test the function and print the result
result = total_commission(sales_rep_name="David", start_date="2023-04-01", end_date="2023-06-30")
print(f"total_commission(sales_rep_name='David', start_date='2023-04-01', end_date='2023-06-30') = {result:.2f}")

total_commission(sales_rep_name='David', start_date='2023-04-01', end_date='2023-06-30') = 89540.00


In [17]:
# Test the function and print the result
result = total_commission(sales_rep_name="Poppy", start_date="2023-03-01", end_date="2023-5-30")
print(f"total_commission(sales_rep_name='Poppy', start_date='2023-03-01', end_date='2023-5-30') = {result:.2f}")

total_commission(sales_rep_name='Poppy', start_date='2023-03-01', end_date='2023-5-30') = 118190.00


# Unit Tests 

Below I wrote some test cases for the function above to try different input values and make sure function is running without any issues. 

In [20]:
def test_total_commission():
    # Test case 1 - Valid input
    result = total_commission(sales_rep_name="Ian", start_date="2023-01-01", end_date="2023-04-30")
    expected_result = 55350.00
    assert result == expected_result, f"Test case 1 failed: expected {expected_result:.2f} but got {result:.2f}"

    # Test case 2 - Non-existing sales rep
    result = total_commission(sales_rep_name="Non-existing Sales Rep", start_date="2023-01-01", end_date="2023-04-30")
    expected_result = None 
    assert result == expected_result, f"Test case 2 failed: expected {expected_result:.2f} but got {result:.2f}"

        
    # Test case 3 - Non-existing date range
    result = total_commission(sales_rep_name="Ian", start_date="2024-01-01", end_date="2024-04-30")
    expected_result = 0
    assert result == expected_result, f"Test case 3 failed: expected {expected_result:.2f} but got {result:.2f}"


    # Test case 4 - Invalid start date
    result = total_commission(sales_rep_name="Ian", start_date="Invalid date", end_date="2023-04-30")
    expected_result = None
    assert result == expected_result, f"Test case 5 failed: expected {expected_result} but got {result}"

    # Test case 5 - Invalid end date
    result = total_commission(sales_rep_name="Ian", start_date="2023-01-01", end_date="Invalid date")
    expected_result = None
    assert result == expected_result, f"Test case 6 failed: expected {expected_result} but got {result}"

    print("All test cases passed")


In [21]:
#call test_total_commission function above
test_total_commission()


Non-existing Sales Rep is not a valid sales rep name.
The start date or end date is not a valid date.
The start date or end date is not a valid date.
All test cases passed


# Error Handling

The function below is the advanced version of the previous function. For example, it will ensure that the datasets are available and have the correct format, check if the sales rep name is valid or not, and so on(check comments in the code for more information).

In [22]:
def total_commission_with_error_handling(sales_rep_name, start_date, end_date):
    # Load the datasets as dataframes
    try:
        with open('Deals.json') as f:
            deals = pd.read_json(f)
        with open('Products.json') as f:
            products = pd.read_json(f)
    except FileNotFoundError:
        print("One or both of the data files are missing.")
        return None
    except ValueError:
        print("One or both of the data files have an invalid format.")
        return None
    
    # Check if the sales rep name is valid
    if sales_rep_name not in deals['sales_rep_name'].unique():
        print(f"{sales_rep_name} is not a valid sales rep name.")
        return None
    
    # Check if the start date and end date are valid dates
    try:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    except ValueError:
        print("The start date or end date is not a valid date.")
        return None
    
    # Check if the start date is before the end date
    if start_date > end_date:
        print("The start date must be before the end date.")
        return None
    
    # Filter the deals dataframe based on the sales rep name and date range
    deals = deals[(deals['sales_rep_name'] == sales_rep_name) & (deals['date'] >= start_date) & (deals['date'] <= end_date)]

    # Merge the filtered deals dataframe with the products dataframe
    merged = pd.merge(deals, products, left_on='product_id', right_on='id', how='left')

    # Calculate the commission for each sale
    merged['commission'] = merged['quantity_products_sold'] * merged['product_amount'] * merged['commission_rate']

    # Group the merged dataframe by sales rep name and sum the commissions
    result = merged.groupby('sales_rep_name')['commission'].sum().get(sales_rep_name, 0)

    return result


### Call "total_commission_with_error_handling" Function 

In [23]:
#example 1: All inputs are valid
total_commission_with_error_handling(sales_rep_name="Ian", start_date="2023-01-01", end_date="2023-04-30")

55350.0

In [24]:
#example 2: sales_rep_name is not valid
total_commission_with_error_handling(sales_rep_name="Saba", start_date="2023-01-01", end_date="2023-04-30")

Saba is not a valid sales rep name.


In [25]:
#example 3: start_date is greater than end_date
total_commission_with_error_handling(sales_rep_name="Ian", start_date="2025-01-01", end_date="2023-04-30")

The start date must be before the end date.
